1. Loading the dataset

In [1]:
# 1.1. Launch commands to automatically reload modules
%load_ext autoreload
%autoreload 2

In [2]:
# 1.2. Import the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt

In [3]:
# 1.3. Load the training, validation and test datasets into a dataframes 
# Load the Parquet files into DataFrames
X_train = pd.read_parquet('../data/processed/X_train_resampled.parquet')
X_val = pd.read_parquet('../data/processed/X_val.parquet')
y_train = pd.read_parquet('../data/processed/y_train_resampled.parquet')
y_val = pd.read_parquet('../data/processed/y_val.parquet')
X_test = pd.read_parquet('../data/processed/X_test_scaled.parquet')

In [4]:
# taking out the player id out of the test data
player_id_test = X_test.pop('player_id')

In [5]:
y_val = y_val.iloc[:, 0]  # Convert the single column DataFrame to a Series
y_val.shape

(9324,)

In [6]:
y_train = y_train.iloc[:, 0]  # Convert the single column DataFrame to a Series
y_train.shape

(73904,)

3. Support Vector Machines - Modelling

In [7]:
#3.1 Import SVM, pi and instantiate the model
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV
from sklearn.pipeline import Pipeline

# Initialize the SVC model
svc = SVC(kernel='linear', probability=True)  # Set probability=True to enable probability estimates


In [8]:
# Initialize RFECV with SVC and cross-validation strategy
rfecv = RFECV(
    estimator=svc, 
    step=1, 
    cv=5,      # Use cv=5 for 5-fold cross-validation
    scoring='roc_auc', 
    n_jobs=-1  # Use all available CPU cores for parallel processing
)


In [9]:
# Create a pipeline with RFECV and SVC
pipeline = Pipeline([
    ('feature_selection', rfecv),
    ('classification', svc)
])

In [10]:
# 3.2 Train the model and save the predicted values
pipeline.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
# Select the probability for the positive class (class 1)
y_train_probs_svc1 = pipeline.predict_proba(X_train)[:, 1]
y_val_probs_svc1 = pipeline.predict_proba(X_val)[:, 1]

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_auc_score, classification_report
# Predict class labels
y_val_pred = pipeline.predict(X_val)

# Print classification report
print(classification_report(y_val, y_val_pred))

# Print selected features
selected_features = X_train.columns[rfecv.support_]
print("Selected features:", selected_features)

In [ ]:
# 3.3 To plot ROC curve and the AUC metric - for comparison of model performances
#To plot ROC curve and the AUC metric

# Set up plotting area
plt.figure(0).clf()

# SVM - Train
fpr, tpr, _ = metrics.roc_curve(y_train, y_train_probs_svc1)
auc = round(metrics.roc_auc_score(y_train, y_train_probs_svc1), 4)
plt.plot(fpr, tpr, label="SVM Train, AUC="+str(auc))

# SVM - Validation
fpr, tpr, _ = metrics.roc_curve(y_val, y_val_probs_svc1)
auc = round(metrics.roc_auc_score(y_val, y_val_probs_svc1), 4)
plt.plot(fpr, tpr, label="SVM Val, AUC="+str(auc))

# Add legend
plt.legend()

# Show the plot
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.show()

In [ ]:
# 3.4 Save the probability of the test data
y_test_probs_svc1 = pipeline.predict_proba(X_test)[:, 1]


In [ ]:
# 3.5 add the 'player_id'column  into the prediction probability
# Convert y_test_prob_rf3 (which is a NumPy array) to a DataFrame
y_test_probs_df = pd.DataFrame(y_test_probs_svc1, columns=['drafted'])

# Concatenate player_id_test and y_test_prob_df along the columns
Results = pd.concat([player_id_test, y_test_probs_df], axis=1)

In [ ]:
Results

In [ ]:
#3.6 save the probability as CSV file

# Convert to DataFrame and save
pd.DataFrame(Results).to_csv('../data/external/SVM_RFECV_Results.csv', index=False)

#3.7 save the model
# Import dump from joblib
from joblib import dump

# 3.12 Save the model and call the files respectively SVM_rfecv.joblib 
dump(svc, '../models/SVM_rfecv.joblib')
